In [1]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

d:\Learning_Aug_2025\final_gen_ai\AgenticAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = TextLoader('langchain_crewai_dataset.txt')
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
llm = ChatOpenAI(model='o4-mini')
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings)
retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k':4, "lambda_mult": 0.7})


In [3]:
len(chunks)

241

In [4]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EF04C0C790>, search_type='mmr', search_kwargs={'k': 4, 'lambda_mult': 0.7})

In [5]:
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question:
"{question}"

Sub-questions:
""")

decomposition_chain = decomposition_prompt | llm | StrOutputParser()
decomposition_chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.\n\nQuestion:\n"{question}"\n\nSub-questions:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EF7FC40B50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EF01203B50>, root_client=<openai.OpenAI object at 0x000001EF7FC42850>, root_async_client=<openai.AsyncOpenAI object at 0x000001EF012034D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [6]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question = decomposition_chain.invoke({"question": query})
print(decomposition_question)

1. What memory architectures and storage mechanisms does LangChain use (e.g. conversational, summary, or vector-store memory)?  
2. How does CrewAI implement and manage memory in its framework?  
3. What kinds of agents (tool-using, planner, router, etc.) does LangChain provide and how do they orchestrate tasks?  
4. How do CrewAI’s agent designs and orchestration patterns differ from those in LangChain?


In [7]:
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.
                                         
Context:
{context}
                                         
Question:
{input}
                                         
""")

qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)
qa_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nUse the context below to answer the question.\n\nContext:\n{context}\n\nQuestion:\n{input}\n\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EF7FC40B50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EF01203B50>, root_client=<openai.OpenAI object at 0x000001EF7FC42850>, root_async_client=<openai.AsyncOpenAI object at 0x000001EF012034D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [10]:
def full_query_decomposition_rag_pipeline(user_query):

    sub_qs_txt = decomposition_chain.invoke({"question": user_query})
    sub_question = [q.strip("-1234567890. ").strip() for q in sub_qs_txt.split("\n") if q.strip()]

    result = []

    for subq in sub_question:
        docs = retriever.invoke(subq)
        response = qa_chain.invoke({"input": subq, "context": docs})
        result.append(f"Q: {subq}\nA: {response}")

    return"\n\n".join(result)


In [12]:
query = "How does LangChain use memory and agents compared to CrewAI?"
response = full_query_decomposition_rag_pipeline(query)

print(response)

Q: Here are four focused sub-questions to guide your document retrieval:
A: Below are four targeted sub‐questions you can use to pull back the most relevant documents or knowledge snippets from your store:

1. How does “knowledge fetching and injection” work in version 4—what mechanisms are used to select, filter, and embed external knowledge into the LLM prompt, and what impact does that have on answer accuracy and hallucination rates?  
2. What architecture and algorithms power LangChain’s hybrid retrieval in v4—specifically, how are sparse (e.g. BM25) and dense (embedding) methods combined, and what recall/precision trade-offs or performance gains have been observed?  
3. In what ways does CrewAI orchestrate specialized agents for multi-step workflows (e.g. market research, legal analysis, product dev, coding), and what coordination, hand-off, or chaining patterns does it employ to boost overall throughput and quality?  
4. Which external tools (web search, calculators, code executi